# Solving Sudoku Puzzles
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**You must implement your own work.** You can and are encouraged to use helper libraries like `numpy`, but you must not use 3rd-party libraries or code that solves the assignment for you. *Even if it is properly referenced*, you only get marks for work you have done. If you are unsure, ask on the forum or ask a tutor directly.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can solve sudoku puzzles. You should be familiar with sudoku puzzles from the unit material. You are given a 9x9 grid with some fixed values. To solve the puzzle, the objective is to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>

For this assignment you will need to submit:
1. The implementation for an agent which can solve sudoku puzzles – this notebook
 * You can use any algorithm you like, from the unit material or otherwise
 * Your code will be subject to automated testing, from which grades will be assigned based on whether it can solve sudokus of varying difficulty
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests included below, make sure your code is compatible with the format of these tests
2. A text file that explains your approach and the decisions you made in your own words – a readme file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * You may write your file in plain text (.txt) or [Markdown](https://www.markdownguide.org/basic-syntax/) (.md)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section

### Choice of Algorithm
The choice of algorithm to solve sudoku puzzles is up to you. We expect you will use search techniques from the unit, but you could make something up yourself, or do some independent research to find something else. You will need to evaluate and balance the trade-off between how well suited you think the algorithm is and how difficult it is to write, but there is some advice below.

You should also consider what *data structures* you use within your code. Certain algorithms will work much faster with certain data structures, and understanding this is considered part of understanding the algorithm itself. 

I suggest you implement *constraint satisfaction* as it is described in the unit material. A good implementation of a backtracking depth-first search with constraint propagation should be sufficient to get a good grade in the automated tests (roughly 60-70%). There is an example of constraint satisfaction on the eight-queens puzzle, and you could use this as a guide to help you get started.

You could also write a successful agent that uses the other search techniques you have seen in the unit so far: basic search, heuristic search, or local search. You may find these easier to implement, though they may perform less well. 

To get a high grade on this assignment will require a particularly efficient implementation of constraint satisfaction, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (readme file), but only the code in this notebook will be used in the automated testing.

## Sample Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [ ]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [1]:
import copy
import numpy as np
import collections
from statistics import mean

class PartialSudokuState:
    def __init__(self, state, n=9):
        self.final_values = state  # final value to hold the numpy array of initial values
        self.n = n
        self.matrix = {}  # Sparse matrix to hold the constraints and values
        self.sol_matrix = {}  # To add and remove the solutions temporarily until goal is reached
        self.block_cords = [(0, 0), (0, 3), (0, 6), (3, 0), (3, 3), (3, 6), (6, 0), (6, 3),
                            (6, 6)]  # Starting coordinate of each blocks
        self.setup_initial_matrix()  # populate the initial matrix
        self.update_initial_matrix_values()  # Remove the constraints from the final matrix

    def setup_initial_matrix(self):
        """
        Populates the initial matrix by the following constraints:
        There are four kinds of constraints:
            Row-Column: Each intersection of a row and column, i.e, each cell, must contain exactly one number.
            Row-Number: Each row must contain each number exactly once
            Column-Number: Each column must contain each number exactly once.
            Box-Number: Each box must contain each number exactly once.
        """
        cell_list = [('cell', (i, j)) for i in range(0, 9) for j in range(0, 9)]  # List of all possible cell and
        # value combination sudoku can have
        row_list = [('row', (i, j)) for i in range(0, 9) for j in
                    range(1, 10)]  # List of all possible row and value combination sudoku can have
        col_list = [('col', (i, j)) for i in range(0, 9) for j in
                    range(1, 10)]  # List of all possible column and value combination sudoku can have
        block_list = [('block', (i, j)) for i in range(0, 9) for j in
                      range(1, 10)]  # List of all possible block and value combination sudoku can have
        self.matrix = collections.defaultdict(set)  # Defines the values of the dictionary as an empty set
        # Creates (row, col, value) for each of the constraints
        for constraint in cell_list:
            for i in range(1, 10):
                self.matrix[constraint].add((constraint[1][0], constraint[1][1], i))
        for constraint in row_list:
            for i in range(0, 9):
                self.matrix[constraint].add((constraint[1][0], i, constraint[1][1]))
        for constraint in col_list:
            for i in range(0, 9):
                self.matrix[constraint].add((i, constraint[1][0], constraint[1][1]))
        for constraint in block_list:
            i = constraint[1][0]
            for y in range(3):
                for x in range(3):
                    self.matrix[constraint].add(
                        (self.block_cords[i][0] + y, self.block_cords[i][1] + x, constraint[1][1]))

    def update_initial_matrix_values(self):
        """
        Iterates through each value in the given puzzle and removes the relevant constraint and values
        """
        for row in range(9):
            for col in range(9):
                value = self.final_values[row][col]
                if value != 0:  # The cell is picked up only if it is not empty (has a value filled in)
                    self.update_related_rcv_constraints((row, col, value))

    def update_related_rcv_constraints(self, rcv):
        """
        Gets a rcv value and removes the all the possible combinations of cell, row, column and block constraint the
        given rcv triplet affects
        """
        row, col, value = rcv  # Unpacks the given rcv triplet

        def delete_cell_rows():
            for i in range(9):
                temp_values = self.matrix.get(('cell', (row, i)))  # Gets the values relevant to all cells in the
                # current row
                if temp_values is not None and (row, i, value) in temp_values:
                    temp_values.remove((row, i, value))
                    self.matrix[('cell', (row, i))] = temp_values

        def delete_cell_col():
            for i in range(9):
                temp_values = self.matrix.get(('cell', (i, col)))
                if temp_values is not None and (i, col, value) in temp_values:
                    temp_values.remove((i, col, value))
                    self.matrix[('cell', (i, col))] = temp_values

        def delete_cell_block(block_cord):
            for i in range(block_cord[0], block_cord[0] + 3):
                for j in range(block_cord[1], block_cord[1] + 3):
                    temp_values = self.matrix.get(('cell', (i, j)))
                    if temp_values is not None and (i, j, value) in temp_values:
                        temp_values.remove((i, j, value))
                        self.matrix[('cell', (i, j))] = temp_values

        def row_remove_possible_cords():
            filtered_dict = dict(filter(lambda item: 'row' in item[0], self.matrix.items()))
            dict_keys = list(filtered_dict.keys())
            for key in dict_keys:
                temp_values = self.matrix.get(key)
                if temp_values is not None and (row, col, key[1][1]) in temp_values:
                    temp_values.remove((row, col, key[1][1]))
                    self.matrix[key] = temp_values

                if temp_values is not None and (key[1][0], col, value) in temp_values:
                    temp_values.remove((key[1][0], col, value))
                    self.matrix[key] = temp_values

                for temp_value in temp_values.copy():
                    (temp_col, temp_row) = map(lambda coord: (coord // 3) * 3, (temp_value[1], temp_value[0]))
                    (rcv_col, rcv_row) = map(lambda coord: (coord // 3) * 3, (col, row))
                    if (temp_col, temp_row) == (rcv_col, rcv_row):
                        if temp_values is not None and (temp_value[0], temp_value[1], value) in temp_values:
                            temp_values.remove((temp_value[0], temp_value[1], value))
                            self.matrix[key] = temp_values

        def col_remove_possible_cords():
            filtered_dict = dict(filter(lambda item: 'col' in item[0], self.matrix.items()))
            dict_keys = list(filtered_dict.keys())
            for key in dict_keys:
                temp_values = self.matrix.get(key)
                if temp_values is not None and (row, col, key[1][1]) in temp_values:
                    temp_values.remove((row, col, key[1][1]))
                    self.matrix[key] = temp_values

                if temp_values is not None and (row, key[1][0], value) in temp_values:
                    temp_values.remove((row, key[1][0], value))
                    self.matrix[key] = temp_values

                for temp_value in temp_values.copy():
                    (temp_col, temp_row) = map(lambda coord: (coord // 3) * 3, (temp_value[1], temp_value[0]))
                    (rcv_col, rcv_row) = map(lambda coord: (coord // 3) * 3, (col, row))
                    if (temp_col, temp_row) == (rcv_col, rcv_row):
                        if temp_values is not None and (temp_value[0], temp_value[1], value) in temp_values:
                            temp_values.remove((temp_value[0], temp_value[1], value))
                            self.matrix[key] = temp_values

        def block_remove_possible_cords():
            col_blocks = [(0, 3, 6), (0, 3, 6), (0, 3, 6), (1, 4, 7), (1, 4, 7), (1, 4, 7), (2, 5, 8), (2, 5, 8),
                          (2, 5, 8)]
            row_blocks = [(0, 1, 2), (0, 1, 2), (0, 1, 2), (3, 4, 5), (3, 4, 5), (3, 4, 5), (6, 7, 8), (6, 7, 8),
                          (6, 7, 8)]
            filtered_dict = dict(filter(lambda item: 'block' in item[0], self.matrix.items()))
            dict_keys = list(filtered_dict.keys())
            for key in dict_keys:
                temp_values = self.matrix.get(key)
                if temp_values is not None and (row, col, key[1][1]) in temp_values:
                    temp_values.remove((row, col, key[1][1]))
                    self.matrix[key] = temp_values

                for i in range(9):
                    if temp_values is not None and (i, col, value) in temp_values:
                        temp_values.remove((i, col, value))
                        self.matrix[key] = temp_values

                    if temp_values is not None and (row, i, value) in temp_values:
                        temp_values.remove((row, i, value))
                        self.matrix[key] = temp_values

        # cell elimination
        delete_cell_rows() # cell - row
        delete_cell_col() # cell - col
        # Round pos down to the nearest multiple of block_size (get top left position of block)
        (coly, rowx) = map(lambda coord: (coord // 3) * 3, (col, row))
        delete_cell_block((rowx, coly)) # cell - block
        # i - cell elimination
        if self.matrix is not None and ('cell', (row, col)) in self.matrix:
            self.matrix.pop(('cell', (row, col)))

        # ii - row elimination
        if self.matrix is not None and ('row', (row, value)) in self.matrix:
            self.matrix.pop(('row', (row, value)))

        # iii - col elimination
        if self.matrix is not None and ('col', (col, value)) in self.matrix:
            self.matrix.pop(('col', (col, value)))

        # iv - block elimination
        block_index = self.block_cords.index((rowx, coly))
        if self.matrix is not None and ('block', (block_index, value)) in self.matrix:
            self.matrix.pop(('block', (block_index, value)))

        # v - remove row possible vals
        row_remove_possible_cords()
        # vi - remove col possible vals
        col_remove_possible_cords()
        # vii - remove block possible vals
        block_remove_possible_cords()


    def pick_constraint(self):
        consts = [v for v in self.matrix if self.matrix[v]]
        # Check there are constraints left
        if not consts:
            # self.solvable = False
            return
        # Get constraint with the shortest number of possible RCVs
        result = min(consts, key=lambda k: len(self.matrix[k]))
        return result

    def set_value(self, rcv):
        row, col, value = rcv
        self.sol_matrix[(row, col)] = value
        self.update_related_rcv_constraints(rcv)

    def is_goal(self):
        if not self.matrix:
            for y, x in self.sol_matrix.keys():
                self.final_values[y, x] = self.sol_matrix[y, x]
            return True
        return False

    def __deepcopy__(self, memodict={}):
        """
        Perform a deepcopy on SudokuState.
        IMPORTANT: New fields added to the class SudokuState also need to be added here.
        :return: New object
        """
        cls = self.__class__
        state = cls.__new__(cls)
        state.final_values = self.final_values
        state.matrix = {k: set(self.matrix[k]) for k in self.matrix.keys()} 
        state.block_cords = self.block_cords
        state.sol_matrix = self.sol_matrix
        return state

In [2]:
def algorithm_x(partial_state):
    constraint = partial_state.pick_constraint()  # Return an uncovered column with the minimal number of rows.
    if constraint is None:
        return None
    values_for_constraint = partial_state.matrix[constraint]
    for value in values_for_constraint.copy():
        copied_state = copy.deepcopy(partial_state)  # Captures the copy of the object to go back to previous state if
        # it didn't lead to a solution
        partial_state.set_value(value)  # Remove the rcv and add it to the sol matrix
        if partial_state.is_goal():
            return partial_state  # if the current state is goal that is returned
        # continue trying with the rcv triples:
        deep_partial_state = algorithm_x(partial_state)
        if deep_partial_state is not None:
            return deep_partial_state # back tracking lead to a partial success
        partial_state = copied_state # The rcv triples doesn't lead to a solved sudoku goes back to the previous state

In [5]:
def sudoku_solver(sudoku):
    """
    Solves a Sudoku puzzle and returns its unique solution.

    Input
        sudoku : 9x9 numpy array
            Empty cells are designated by 0.

    Output
        9x9 numpy array of integers
            It contains the solution, if there is one. If there is no solution, all array entries should be -1.
    """
    error_grid = np.array([[-1 for _ in range(9)] for _ in range(9)])
    partial_state = PartialSudokuState(sudoku)
    goal = algorithm_x(partial_state)
    if goal is None:
        return error_grid
    else:
        return goal.final_values

All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up – many more than 15 of each. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 20 seconds each on the test machine to count as successful – if it takes longer, it will timeout. Note that this is a maximum, not a goal. Higher grades require better performance on harder puzzles. As a very rough benchmark, you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms and data structures. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [8]:
SKIP_TESTS = True

def tests():
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break
            
if not SKIP_TESTS:
    tests()

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [9]:
def submission_tests():
    import sys
    import pathlib

    fail = False;

    if not SKIP_TESTS:
        fail = True;
        print("You must set the SKIP_TESTS constant to True in the cell above.")

    p1 = pathlib.Path('./readme.txt')
    p2 = pathlib.Path('./readme.md')
    if not (p1.is_file() or p2.is_file()):
        fail = True;
        print("You must include a separate file called readme.txt or readme.md in your submission.")

    p3 = pathlib.Path('./sudoku.ipynb')
    if not p3.is_file():
        fail = True
        print("This notebook file must be named sudoku.ipynb")

    if "sudoku_solver" not in globals():
        fail = True;
        print("You must include a function called sudoku_solver which accepts a numpy array.")
    else: 
        sudoku = np.load("data/very_easy_puzzle.npy")[0]
        solution = np.load("data/very_easy_solution.npy")[0]

        if not np.array_equal(sudoku_solver(sudoku), solution):
            print("Warning:")
            print("Your sudoku_solver function does not correctly solve the first sudoku.")
            print()
            print("Your assignment is unlikely to get any marks from the autograder. While we will")
            print("try to check it manually to assign some partial credit, we encourage you to ask")
            print("for help on the forum or directly to a tutor.")
            print()
            print("Please use the readme file to explain your code anyway.")

    if fail:
        print()
        sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
    else:
        print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
        print("assignment page, without changing any filenames.")
        print()
        print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")
        
submission_tests()

All checks passed. When you are ready to submit, upload the notebook and readme file to the
assignment page, without changing any filenames.

If you need to submit multiple files, you can archive them in a .zip file. (No other format.)


In [ ]:
# This is a TEST CELL. Do not delete or change.